#### Lets import all the required libraries first.

In [18]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup # library for scraping from a website

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

!pip install geocoder
import geocoder # to get longitude and latitude

#### Extract the data from Wikipedia page and create Dataframe.

In [19]:
# Get the webpage
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text

# Extract only the table
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_='sortable')

# Get the values from the Wikipedia table and store into a dataframe
row = [] # initialize row list

for tr in table.find_all('tr'):                                           
    if tr.find_all('th') == []:                                           
        row.append([td.get_text(strip=True) for td in tr.find_all('td')]) 

# Assign columns names and turn row into a dataframe of neighborhoods
column_names = ['Postcode', 'Borough', 'Neighborhood']

neighborhoods_raw = pd.DataFrame(row, columns=column_names) # create a raw table of neighborhoods
neighborhoods_raw.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


#### Rows containing 'Not assigned' boroughs are dropped.

In [38]:
drop_index = neighborhoods_raw[neighborhoods_raw['Borough'] == 'Not assigned'].index 
neighborhoods = neighborhoods_raw.drop(drop_index, axis=0)                           
neighborhoods.reset_index(drop=True, inplace=True)                                   

#### Assign borough names to neighborhoods when neighborhood is 'Not assigned' and Neighborhoods with a same postcode are merged into a single cell, separated by commas.

In [39]:
nh_na = neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned'].index
neighborhoods.iloc[nh_na, 2] = neighborhoods['Borough'][nh_na]

In [40]:
neighborhoods = neighborhoods.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))
neighborhoods.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
neighborhoods.shape

(103, 3)

### 2. Longitude and latitude of the neighborhoods

In [42]:
# Initialize varialbes
lat = []
lng = []
lat_lng_coords = None

# Get postcodes from neighborhoods table
postal_code = neighborhoods['Postcode']

# Store latitude and longitude values in lat and lng
for pc in postal_code:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])

#### Add the lists containing latitude and longitude values to the neighborhood dataframe.


In [43]:
nh_complete = neighborhoods
nh_complete['Latitude'] = lat
nh_complete['Longitude'] = lng

In [44]:
nh_complete.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


### 3. Explore and cluster the neighborhoods.


In [28]:
# New dataframe with only the original city included
toronto = nh_complete[nh_complete['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
toronto.shape

(39, 5)

In [29]:
# Get the latitude and longitude of Toronto
g = geocoder.arcgis('Toronto, Ontario')
lat_tor = g.latlng[0]
lng_tor = g.latlng[1]

# Create a map of Toronto
map_toronto = folium.Map(location=[lat_tor, lng_tor], zoom_start=11)

# Add markers to map
for lat, lng, bor, postcode in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Postcode']):
    label = '{}, {}'.format(postcode, bor)        # popup labels with postcode and borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)  
# Lets print map of Toronto
map_toronto

In [49]:
CLIENT_ID = 'JW1LOUKCHOBJJGPYYZFROQZINFPN0AU4E44WNXM51NPILF3W' # your Foursquare ID
CLIENT_SECRET = 'QEEXIE4ODGP1UZVI35RD0OQL2XS2WQWE3ZNAQ3T4CMSOMZNJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JW1LOUKCHOBJJGPYYZFROQZINFPN0AU4E44WNXM51NPILF3W
CLIENT_SECRET:QEEXIE4ODGP1UZVI35RD0OQL2XS2WQWE3ZNAQ3T4CMSOMZNJ


In [50]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


#### Finding Indian restaurants in Toronto original city and plotting them on the map.

In [52]:
search_query = 'Indian'
radius = 500
print(search_query + ' .... OK!')

Indian .... OK!


In [53]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JW1LOUKCHOBJJGPYYZFROQZINFPN0AU4E44WNXM51NPILF3W&client_secret=QEEXIE4ODGP1UZVI35RD0OQL2XS2WQWE3ZNAQ3T4CMSOMZNJ&ll=43.653963,-79.387207&v=20180604&query=Indian&radius=500&limit=30'

In [55]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df795dcdff8e6001bbcdfec'},
 'response': {'venues': [{'id': '4afd920ff964a520ad2822e3',
    'name': 'Indian Biriyani House',
    'location': {'address': '181 Dundas St W',
     'crossStreet': 'W of Chestnut St',
     'lat': 43.65511996683289,
     'lng': -79.3866447102921,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65511996683289,
       'lng': -79.3866447102921}],
     'distance': 136,
     'postalCode': 'M5G 1C7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['181 Dundas St W (W of Chestnut St)',
      'Toronto ON M5G 1C7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1576506866',
  

In [57]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
indian_rests = json_normalize(venues)
indian_rests.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4afd920ff964a520ad2822e3,Indian Biriyani House,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1576506866,False,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",136,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O..."
1,4b2a634af964a52020a824e3,Indian Flavour,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1576506866,False,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",311,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront..."
2,596f9dea1fa7632be2eacab9,Mami's Indian Cuisine,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1576506866,False,NaN,NaN,43.656986,-79.385840,"[{'label': 'display', 'lat': 43.65698561897954...",354,M5G 2N2,CA,Toronto,ON,Canada,"[Toronto ON M5G 2N2, Canada]"
3,4e122e6cb0fbfef99b985de1,Jodpore Club Indian Cuisine,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1576506866,False,NaN,NaN,43.655946,-79.393504,"[{'label': 'display', 'lat': 43.655946, 'lng':...",553,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]"


In [58]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in indian_rests.columns if col.startswith('location.')] + ['id']
dataframe_filtered = indian_rests.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Indian Biriyani House,Indian Restaurant,181 Dundas St W,W of Chestnut St,43.655120,-79.386645,"[{'label': 'display', 'lat': 43.65511996683289...",136,M5G 1C7,CA,Toronto,ON,Canada,"[181 Dundas St W (W of Chestnut St), Toronto O...",4afd920ff964a520ad2822e3
1,Indian Flavour,Indian Restaurant,123 Dundas St W,btw Elizabeth & Bay,43.655649,-79.384119,"[{'label': 'display', 'lat': 43.65564910619165...",311,NaN,CA,Toronto,ON,Canada,"[123 Dundas St W (btw Elizabeth & Bay), Toront...",4b2a634af964a52020a824e3
2,Mami's Indian Cuisine,Food Truck,NaN,NaN,43.656986,-79.385840,"[{'label': 'display', 'lat': 43.65698561897954...",354,M5G 2N2,CA,Toronto,ON,Canada,"[Toronto ON M5G 2N2, Canada]",596f9dea1fa7632be2eacab9
3,Jodpore Club Indian Cuisine,Indian Restaurant,NaN,NaN,43.655946,-79.393504,"[{'label': 'display', 'lat': 43.655946, 'lng':...",553,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",4e122e6cb0fbfef99b985de1


In [59]:
dataframe_filtered.name

0          Indian Biriyani House
1                 Indian Flavour
2          Mami's Indian Cuisine
3    Jodpore Club Indian Cuisine
Name: name, dtype: object

In [60]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Hence we have successfully found and plotted Indian restaurants in Toronto original city.